In [8]:
import pandas as pd
import seaborn as sns
import numpy as np
import json
import sys
# setting path
sys.path.append('../common')
import utils

SETUP_PATH = 'setup.json'
f = open(SETUP_PATH)
data = json.load(f)

In [9]:
# Initialize database
import requests
import datetime

clear_url = data['app']['clearUrl']
print(clear_url)
response = requests.post(clear_url)
print(response.content)
print(response.status_code)

fill_in_url = data['app']['fillInUrl']
N = data['fill_in_quantity']
print(fill_in_url)
print(N)
patch = 10000
iterations = int(N / patch)
print(iterations)
for i in range(1, iterations + 1):
    first_time = datetime.datetime.now()
    response = requests.post(fill_in_url, params={'N': patch})
    later_time = datetime.datetime.now()
    difference = later_time - first_time
    print(f'{i}/{iterations}: {response.content}')
    print(f'{i}/{iterations}: {response.status_code}')
    print(f'{i}/{iterations}: duration: {utils.durationToString(difference.seconds)}')
    remaining_iterations = iterations - i
    print(f'{i}/{iterations}: expected end in: {utils.durationToString(difference.seconds * remaining_iterations)}')

patch = N % patch
if patch > 0:
    response = requests.post(fill_in_url, params={'N': patch})
    print(response.content)
    print(response.status_code)

http://localhost:3000/member/clear/
b''
201
http://localhost:3000/member/fill-in/
7500
0
b'Ok'
201


In [3]:
def run_k6(app: dict, script_path: str, vus: int, dir_name):
    !k6 run {script_path} \
        --env vus={vus} \
        --env url={app["url"]} \
        --out json={dir_name}/{app["name"]}.json \
        --out csv={dir_name}/{app["name"]}.csv

## Run k6 test script

In [4]:
import os
import shutil
from datetime import datetime
import math 



# prepare directory structure
if not os.path.exists(data["test_result_base_dir"]):
    os.makedirs(data["test_result_base_dir"])

dt_string = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
print("date and time =", dt_string)

dir_name = f'./{data["test_result_base_dir"]}/{data["test_result_dir_prefix"]}{dt_string}'
print(dir_name)
if not os.path.exists(dir_name):
    os.makedirs(dir_name)
    
# copy script
shutil.copy(data["script_path"], dir_name)
shutil.copy(SETUP_PATH, dir_name)

out_file_name = "out_fill_in.txt"
out_file = open(out_file_name, "a")
out_file.write(f'\n-------------------------')
out_file.close()

vus = 1 # initial value
beg = 1
end = None
threshold = 0
while (True):
    # run test
    run_k6(data["app"], script_path=data["script_path"], vus=vus, dir_name=dir_name)

    # load dataframe
    dir = utils.get_last_result_dir(setup_data=data)
    path = f'./{dir}/{data["app"]["name"]}.csv'
    df = pd.read_csv(path)

    # calculate metric
    incorrect_part = utils.get_incorrect_part(df)
        
    print(f'Incorrect part for vus = {vus}: {incorrect_part}\n')

    out_file = open(out_file_name, "a")
    out_file.write(f'\nvus = {vus}, beg = {beg}, end = {end}, incorrect_part = {incorrect_part}')
    out_file.close()

    if (incorrect_part > threshold):
        end = vus
        vus = math.floor((end + beg) / 2)
    else:
        beg = vus
        vus = vus * 2 if end == None else math.floor((end + beg) / 2)
    
    if end and end - beg < 2:
        break
        


date and time = 2023-11-04_20-40-36
./out/out_2023-11-04_20-40-36



          /\      |‾‾| /‾‾/   /‾‾/   
     /\  /  \     |  |/  /   /  /    
    /  \/    \    |     (   /   ‾‾\  
   /          \   |  |\  \ |  (‾)  | 
  / __________ \  |__| \__\ \_____/ .io

  execution: local
     script: ../../grafana-k6/script.js
     output: json (./out/out_2023-11-04_20-40-36/nestjs.json), csv (./out/out_2023-11-04_20-40-36/nestjs.csv)

  scenarios: (100.00%) 1 scenario, 1 max VUs, 1m0s max duration (incl. graceful stop):
           * default: 1 looping VUs for 30s (gracefulStop: 30s)


running (0m00.1s), 1/1 VUs, 0 complete and 0 interrupted iterations
default   [--------------------------------------] 1 VUs  00.1s/30s

running (0m00.2s), 1/1 VUs, 0 complete and 0 interrupted iterations
default   [--------------------------------------] 1 VUs  00.2s/30s

running (0m00.3s), 1/1 VUs, 0 complete and 0 interrupted iterations
default   [--------------------------------------] 1 VUs  00.3s/30s

running (0m00.4s), 1/1 VUs, 0 complete and 0 interrupted iterations
defa

/home/jan/Dokumenty/Studia/MSem3/Magisterka/sandbox/framework-benchmark/tests/04-limit-nestjs/../common/utils.py:23: RuntimeWarning: invalid value encountered in scalar divide
  incorrect_part = incorrect_status.count() / (correct_status.count() + incorrect_status.count())



          /\      |‾‾| /‾‾/   /‾‾/   
     /\  /  \     |  |/  /   /  /    
    /  \/    \    |     (   /   ‾‾\  
   /          \   |  |\  \ |  (‾)  | 
  / __________ \  |__| \__\ \_____/ .io

  execution: local
     script: ../../grafana-k6/script.js
     output: json (./out/out_2023-11-04_20-40-36/nestjs.json), csv (./out/out_2023-11-04_20-40-36/nestjs.csv)

  scenarios: (100.00%) 1 scenario, 2 max VUs, 1m0s max duration (incl. graceful stop):
           * default: 2 looping VUs for 30s (gracefulStop: 30s)


running (0m00.1s), 2/2 VUs, 0 complete and 0 interrupted iterations
default   [--------------------------------------] 2 VUs  00.1s/30s

running (0m00.2s), 2/2 VUs, 0 complete and 0 interrupted iterations
default   [--------------------------------------] 2 VUs  00.2s/30s

running (0m00.3s), 2/2 VUs, 0 complete and 0 interrupted iterations
default   [--------------------------------------] 2 VUs  00.3s/30s

running (0m00.4s), 2/2 VUs, 0 complete and 0 interrupted iterations
defa

/home/jan/Dokumenty/Studia/MSem3/Magisterka/sandbox/framework-benchmark/tests/04-limit-nestjs/../common/utils.py:23: RuntimeWarning: invalid value encountered in scalar divide
  incorrect_part = incorrect_status.count() / (correct_status.count() + incorrect_status.count())



          /\      |‾‾| /‾‾/   /‾‾/   
     /\  /  \     |  |/  /   /  /    
    /  \/    \    |     (   /   ‾‾\  
   /          \   |  |\  \ |  (‾)  | 
  / __________ \  |__| \__\ \_____/ .io

  execution: local
     script: ../../grafana-k6/script.js
     output: json (./out/out_2023-11-04_20-40-36/nestjs.json), csv (./out/out_2023-11-04_20-40-36/nestjs.csv)

  scenarios: (100.00%) 1 scenario, 4 max VUs, 1m0s max duration (incl. graceful stop):
           * default: 4 looping VUs for 30s (gracefulStop: 30s)


running (0m00.1s), 4/4 VUs, 0 complete and 0 interrupted iterations
default   [--------------------------------------] 4 VUs  00.1s/30s

running (0m00.2s), 4/4 VUs, 0 complete and 0 interrupted iterations
default   [--------------------------------------] 4 VUs  00.2s/30s

running (0m00.3s), 4/4 VUs, 0 complete and 0 interrupted iterations
default   [--------------------------------------] 4 VUs  00.3s/30s

running (0m00.4s), 4/4 VUs, 0 complete and 0 interrupted iterations
defa